<a href="https://colab.research.google.com/github/TheShiny1/TwitchO7/blob/master/Twitch_Web_Scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kora -q

     |████████████████████████████████| 57 kB 427 kB/s 
     |████████████████████████████████| 1.6 MB 17.0 MB/s 


In [ ]:
import requests as r
import pandas as pd
import re
import pytz
from datetime import datetime
import plotly.graph_objects as go
import time
import json
from bs4 import BeautifulSoup
from kora.selenium import wd

In [ ]:
url = 'https://gql.twitch.tv/gql'
headers="""Accept: */*
Accept-Encoding: gzip, deflate, br
Accept-Language: en-US
Authorization: OAuth qkomnzb7wtwqtlnl5tq9nnfdrxva08
Client-Id: kimne78kx3ncx6brgo4mv6wki5h1ko
Client-Session-Id: 4ce6d655b8336a6e
Client-Version: c7092876-2729-438a-8745-6d9ad439260a
Connection: keep-alive
Content-Length: 7577
Content-Type: text/plain;charset=UTF-8
Host: gql.twitch.tv
Origin: https://www.twitch.tv
Referer: https://www.twitch.tv/
Sec-Fetch-Dest: empty
Sec-Fetch-Mode: cors
Sec-Fetch-Site: same-site
User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.5195.127 Safari/537.36
X-Device-Id: UhXtr0NFXHMIbdLmkGCgHYZEGIaJl8zD"""

In [ ]:
headers=dict(i.split(': ') for i in headers.split('\n'))

In [ ]:
data = """[{"operationName":"UseViewCount","variables":{"channelLogin":"}, "extensions":{"persistedQuery":{"version":1,"sha256Hash":"00b11c9c428f79ae228f30080a06ffd8226a1f068d6f52fbc057cbde66e994c2"}}}]"""

In [ ]:
r.post(url, headers=headers, data=data).json()

{'error': 'Bad Request',
 'status': 400,
 'message': "invalid character 'e' after object key:value pair"}

In [ ]:
def get_payload(name_of_the_channel, get_counts, get_general_info = False, get_preview_image = False, get_stream_info = False):
  x = {"operationName":"UseViewCount","variables":{"channelLogin":name_of_the_channel}, "extensions":{"persistedQuery":{"version":1,"sha256Hash":"00b11c9c428f79ae228f30080a06ffd8226a1f068d6f52fbc057cbde66e994c2"}}}
  y = {"operationName":"PlayerTrackingContextQuery","variables":{"channel":name_of_the_channel,"isLive":True,"hasCollection":False,"collectionID":"","videoID":"","hasVideo":False,"slug":"","hasClip":False},"extensions":{"persistedQuery":{"version":1,"sha256Hash":"3fbf508886ff5e008cb94047acc752aad7428c07b6055995604de16c4b01160a"}}}
  z = {"operationName":"VideoPreviewOverlay","variables":{"login":name_of_the_channel},"extensions":{"persistedQuery":{"version":1,"sha256Hash":"3006e77e51b128d838fa4e835723ca4dc9a05c5efd4466c1085215c6e437e65c"}}}
  t = {"operationName":"StreamMetadata","variables":{"channelLogin":name_of_the_channel},"extensions":{"persistedQuery":{"version":1,"sha256Hash":"a647c2a13599e5991e175155f798ca7f1ecddde73f7f341f39009c14dbf59962"}}}
  pre = """[{},{},{},{}]""".format(str(x)*get_counts, str(y)*get_general_info, str(z)*get_preview_image, str(t)*get_stream_info).replace("'",'"')
  pre = re.sub(',+',',',pre)
  pre = re.sub(',+]$',']',pre)
  return re.sub('^\[','[',pre)

In [ ]:
get_payload('xqc', get_counts=True)

'[{"operationName": "UseViewCount", "variables": {"channelLogin": "xqc"}, "extensions": {"persistedQuery": {"version": 1, "sha256Hash": "00b11c9c428f79ae228f30080a06ffd8226a1f068d6f52fbc057cbde66e994c2"}}}]'

In [ ]:
def clean_a_bit(data: r.models.Response) -> pd.Series(dtype = 'object'):
  data = data.json()
  namedData = dict()
  cleanedData = dict()
  for i in data:
    name = i['extensions']['operationName']
    namedData[name] = i['data']
  now = datetime.now(tz =pytz.timezone('America/New_York'))

  if 'PlayerTrackingContextQuery' in namedData.keys():
    cleanedData['streamer_id'] = namedData['PlayerTrackingContextQuery']['user']['id']
    cleanedData['streamer_login'] = namedData['PlayerTrackingContextQuery']['user']['login']
    cleanedData['game_id'] = namedData['PlayerTrackingContextQuery']['user']['broadcastSetting']['id']
    cleanedData['name'] = namedData['PlayerTrackingContextQuery']['user']['broadcastSetting']['title']
    cleanedData['type'] = namedData['PlayerTrackingContextQuery']['user']['broadcastSetting']['__typename']
  if 'VideoPreviewOverlay' in namedData.keys():
    cleanedData['PreviewImage'] = namedData['VideoPreviewOverlay']['user']['stream']['PreviewImage']
  if 'StreamMetaData' in namedData.keys():
    cleanedData['lastBroadcast_title'] = namedData['StreamMetaData']['user']['lastBroadcast']['title']
    cleanedData['current_created_at'] = namedData['StreamMetaData']['user']['stream']['createdAt']
  if 'UseViewCount' in namedData.keys():
    cleanedData['current_viewers'] = namedData['UseViewCount']['user']['stream']['viewersCount']
  series = pd.Series(cleanedData)
  series['time']=now
  return series.to_frame().T.set_index('time')

In [ ]:
def get_data(username, get_counts, get_general_info=False, get_preview_image=False, get_stream_info=False, **kwargs):
  url = 'https://gql.twitch.tv/gql'
  headers = """Accept: */*
Accept-Encoding: gzip, deflate, br
Accept-Language: en-US
Authorization: OAuth m40f8ps9azcfwbqftpbbbkfinlvc4z
Client-Id: kimne78kx3ncx6brgo4mv6wki5h1ko
Client-Session-Id: 44767b0614275039
Client-Version: 19cbb0be-64ad-46bd-a718-ec9fb15d3e0a
Connection: keep-alive
Content-Length: 7662
Content-Type: text/plain;charset=UTF-8
Host: gql.twitch.tv
Origin: https://www.twitch.tv
Referer: https://www.twitch.tv/
Sec-Fetch-Dest: empty
Sec-Fetch-Mode: cors
Sec-Fetch-Site: same-site
User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.5195.127 Safari/537.36
X-Device-Id: 9fe67639f4e1bc52""".format(username)
  headers = dict(i.split(': ') for i in headers.split('\n'))
  resp=r.post(url, headers=headers, data=get_payload(username,get_counts,get_preview_image,get_stream_info))
  return clean_a_bit(resp)

In [ ]:
get_data('xqc', get_counts=True)

,current_viewers
time,
2022-10-31 11:26:36.774219-04:00,63759


In [ ]:
wd.get("https://twitchtracker.com/xqc")
html_source = wd.page_source
print(html_source)

<html lang="en"><head>
<meta charset="utf-8">
<meta http-equiv="X-UA-Compatible" content="IE=edge">
<meta name="viewport" content="width=device-width, initial-scale=1.0">
<title>xQc - Streamer Overview &amp; Stats · TwitchTracker</title>
<meta name="description" content="Overview of xqc activities, statistics, played games and past streams.">
<meta name="keywords" content="twitch, statistics, stream, channel, stats, games, subs, subscribers, clips, viewers, followers, views, shroud, ninja, drdisrespect, summit1g, lirik, dreadztv">

<script src="https://partner.googleadservices.com/gampad/cookie.js?domain=twitchtracker.com&amp;callback=_gfp_s_&amp;client=ca-pub-3581129455196386&amp;gpid_exp=1"></script><script src="https://pagead2.googlesyndication.com/pagead/managed/js/adsense/m202211030101/show_ads_impl.js?bust=31070723" id="google_shimpl"></script><script async="" src="https://pagead2.googlesyndication.com/pagead/js/adsbygoogle.js?client=ca-pub-3581129455196386" crossorigin="anonymou

In [ ]:
soup = BeautifulSoup(html_source, 'lxml')
values = soup.find_all('div', class_='g-x-s-value g-x-s-contrast')[0].find("span").text
print(values)

NameError: ignored

In [ ]:
data = get_data('xqc', get_counts=True)
def create_plot(data):
  fig = go.FigureWidget([go.Scatter(x = data.index, y = data.current_viewers,
                                    mode = 'lines')])
  return fig
fig = create_plot(data)
fig.show()

In [ ]:
def update(data, fig):
  dat = create_plot(data).data
  fig.data[0].x = dat[0].x
  fig.data[0].y = dat[0].y

In [ ]:
while True:
  df = get_data('xqc', get_counts=True)
  data = data.append(df)
  time.sleep(20)
  update(data,fig)
  print('Done!')

Done!


KeyboardInterrupt: ignored

In [ ]:
fig.show()